<center> <h1>Workshop: Diff-in-diff</h1> </center> 
<center> <h2>Application: Soda Taxes</h2> </center> 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

### 0. Setting
Soda taxes have become a popular tool to fight obesity in the US. You are analyzing data from Philadelphia, which implemented such a tax and you are trying to figure out whether the tax was successful in lowering soda sales. You observe average weekly sales and prices for soda across a large set of retail stores (identified by the variable store id) inside and outside of Philadelphia before and after the tax (i.e. in two time-periods). Note that soda is only taxed in stores in Philadelphia, but not in stores outside of the city. Use the data-set soda_data.csv throughout this workshop.


### 1. Difference-in-differences

#### 1.1. "Manual" approach

**Q:** Add code that calculates average weekly quantity sold for the following four groups of observations:
1. stores in Philadelphia before the tax
2. stores in Philadelphia after the tax
3. Stores outside of the city before the tax
4. stores outside of the city after the tax. 

Finally, add code that computes the difference-in-differences estimator of the soda tax effect and interpret your findings.

In [4]:
soda_data = pd.read_csv('soda_data.csv')
soda_data.head()

,store_id,philly,after_tax_dummy,soda_quantity,soda_price,income_rescale,distance_border_outside
0,1,0,0,722633.489677,2.856049,0.576050,2.658631
1,1,0,1,704771.691700,3.082004,0.576050,2.658631
2,2,0,0,7182.768466,4.415120,1.056932,2.002235
3,2,0,1,7592.831115,4.490781,1.056932,2.002235
4,3,0,0,6505.230771,3.644063,0.972903,6.851919


In [5]:
Y_in_before = np.mean(soda_data[(soda_data.philly==1) & (soda_data.after_tax_dummy==0)].soda_quantity)
print("Y_in_before: ",Y_in_before)
Y_in_after = np.mean(soda_data[(soda_data.philly==1) & (soda_data.after_tax_dummy==1)].soda_quantity)
print("Y_in_after: ",Y_in_after)
Y_out_before = np.mean(soda_data[(soda_data.philly==0) & (soda_data.after_tax_dummy==0)].soda_quantity)
print("Y_out_before: ",Y_out_before)
Y_out_after = np.mean(soda_data[(soda_data.philly==0) & (soda_data.after_tax_dummy==1)].soda_quantity)
print("Y_out_after: ",Y_out_after)
print("DD: ",Y_in_after-Y_in_before - (Y_out_after-Y_out_before))

Y_in_before:  43148.9329654178
Y_in_after:  24750.375832229285
Y_out_before:  75629.56673598864
Y_out_after:  82738.54315164233
DD:  -25507.5335488422


**A:** The effect of the soda tax in the average store is a decrease in average soda sales of 25508 units (weekly sales in oz) more in Philly compared to outside Philly.

#### 1.2. Regression approach

**Q:** Now run a regression that yields the difference-in-difference estimate. Interpret all coefficients (including the intercept). Verify that the result is identical to the one you obtained in the previous section.

In [6]:
DinD_reg = smf.ols(formula = 'soda_quantity ~ philly + after_tax_dummy + philly:after_tax_dummy', data = soda_data).fit()
print(DinD_reg.summary())
print("philly:after_tax_dummy:", DinD_reg.params["philly:after_tax_dummy"])


                            OLS Regression Results                            
Dep. Variable:          soda_quantity   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     22.52
Date:                Thu, 21 Mar 2024   Prob (F-statistic):           2.23e-14
Time:                        01:24:54   Log-Likelihood:                -32325.
No. Observations:                2452   AIC:                         6.466e+04
Df Residuals:                    2448   BIC:                         6.468e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               7.56

**A:** 
    
    - Intercept: is the average sales before the tax outside Philadelphia.
    
    - Philly-dummy coefficient: is the difference in sales in Philadelphia relative to average sales outside of the city before the tax introduction.
    
    - After-tax dummy coefficient: is the difference in sales in stores outside of Philadelphia after the tax relative to before the tax.
    
    - Interaction term coefficient: is the differential change over time in sales in Philadelphia relative to the change over time outside of Philadelphia (i.e. the difference-in-difference estimate).

#### 1.3. Without stores near the city border

We now run the same regression, but excluding from the regression all the stores that are outside Philadelphia and that are less than 6 miles from the city border.

In [7]:
#just checking that this is not making my sample too small
soda_data["no_buffer"] = (soda_data.philly==1) | (soda_data.distance_border_outside>6)
print(np.mean(soda_data.no_buffer))

0.6778140293637847


In [8]:
soda_data[soda_data["no_buffer"]== True].head()

,store_id,philly,after_tax_dummy,soda_quantity,soda_price,income_rescale,distance_border_outside,no_buffer
4,3,0,0,6505.230771,3.644063,0.972903,6.851919,True
5,3,0,1,5360.828900,3.838475,0.972903,6.851919,True
6,4,0,0,23260.225775,3.429815,1.150615,9.713602,True
7,4,0,1,19414.963543,3.536392,1.150615,9.713602,True
12,7,1,0,4434.479611,4.179931,0.293266,0.000000,True


In [9]:
DinD_nobuffer = smf.ols(formula = 'soda_quantity ~ philly + after_tax_dummy + philly:after_tax_dummy', data = soda_data[soda_data["no_buffer"]==True]).fit()
print(DinD_nobuffer.summary())
print("after_tax_dummy:", DinD_nobuffer.params["after_tax_dummy"])
print("philly:after_tax_dummy:", DinD_nobuffer.params["philly:after_tax_dummy"])

                            OLS Regression Results                            
Dep. Variable:          soda_quantity   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     21.51
Date:                Thu, 21 Mar 2024   Prob (F-statistic):           1.14e-13
Time:                        01:25:58   Log-Likelihood:                -21689.
No. Observations:                1662   AIC:                         4.339e+04
Df Residuals:                    1658   BIC:                         4.341e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               7.75

**Q:** Interpret the coefficients on the after tax dummy and the after-tax/Philadelphia interaction term. How do these results differ from the regression you ran in the previous section (and why do they differ)? Which specification do you prefer?

**A:** The control group of stores outside of the city is different in this regression. Instead of an increase, we now find a slight decrease (-898) in sales for stores outside of Philadelphia after the tax goes into effect. Relative to the decrease outside, the decrease inside Philadelphia is larger. The difference-in-difference estimate is smaller, because the Philadelphia change over time (which remains the same) is now compared against a small decrease outside the city rather than a larger increase (see the after-tax dummy coefficient in 1.2). The difference between regressions occurs because stores outside Philly but close to the city are indirectly affected by the tax. For example, because some consumers drive outside of Philadelphia to buy soda there. Because we want the stores in the control group to be unaffected by the tax, the regression which excludes stores close to the city border is preferable.

### 2. Fixed effect regressions

**Q:** Run the same regression as in (1.3), but include a full set of store dummies. (In R you can use the plm() command based on the syntax we used last time. Note that there are no time fixed effects.) Which control variables from the previous specification do you need to maintain, which ones are redundant?

In [10]:
store_FE_reg = smf.ols(formula = "soda_quantity ~ after_tax_dummy + philly:after_tax_dummy + C(store_id)", data=soda_data[soda_data["no_buffer"]==True]).fit()
print(store_FE_reg.summary())

                            OLS Regression Results                            
Dep. Variable:          soda_quantity   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     28.88
Date:                Thu, 21 Mar 2024   Prob (F-statistic):               0.00
Time:                        01:28:57   Log-Likelihood:                -18895.
No. Observations:                1662   AIC:                         3.946e+04
Df Residuals:                     829   BIC:                         4.397e+04
Df Model:                         832                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               6382

In [11]:
store_FE_reg = smf.ols(formula = "soda_quantity ~ after_tax_dummy + philly:after_tax_dummy + C(store_id)", data=soda_data).fit()
print(store_FE_reg.summary())

                            OLS Regression Results                            
Dep. Variable:          soda_quantity   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     21.10
Date:                Thu, 21 Mar 2024   Prob (F-statistic):               0.00
Time:                        01:29:31   Log-Likelihood:                -28560.
No. Observations:                2452   AIC:                         5.958e+04
Df Residuals:                    1224   BIC:                         6.670e+04
Df Model:                        1227                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               7.10

**A:** There is no need to control for the Philadelphia dummy, because we now have a separate fixed effect for each store (some of which are in Philadelphia, some of which are outside of the city). Because the Philadelphia dummy is time-invariant, this variable cannot be estimated anymore once we control for store fixed effects.

### 3. Estimating cross-shopping / leakage effects

**Q:** Extend the specification with store fixed effects from Section 2 (based on all stores) in a way that allows you to answer the following questions: 

(a) How much sales changed in Philadelphia stores after the tax, relative to stores outside Philadelphia that are more than 6 miles away from the city border? 

(b) How much sales changed in stores outside Philadelphia that are within 6 miles of the city border, relative to stores more than 6 miles away? Note that you need to generate a new variable before running this regression. Interpret the relevant regression coefficients. Relate your findings to the difference between the regressions in (1.2) and (1.3).

In [10]:
soda_data["near_philly"] = (soda_data.philly==0) & (soda_data.distance_border_outside<6)

store_FE_full_sample_reg  = smf.ols(formula = "soda_quantity ~ after_tax_dummy + philly:after_tax_dummy + near_philly:after_tax_dummy + C(store_id)", data = soda_data).fit()
print(store_FE_full_sample_reg.summary())

                            OLS Regression Results                            
Dep. Variable:          soda_quantity   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     21.47
Date:                Fri, 11 Feb 2022   Prob (F-statistic):               0.00
Time:                        16:05:02   Log-Likelihood:                -28538.
No. Observations:                2452   AIC:                         5.953e+04
Df Residuals:                    1223   BIC:                         6.667e+04
Df Model:                        1228                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [16]:
soda_data["near_philly"] = (soda_data.philly==0) & (soda_data.distance_border_outside<6)

store_FE_full_sample_reg  = smf.ols(formula = "soda_quantity ~ after_tax_dummy + philly:after_tax_dummy+ near_philly:after_tax_dummy + near_philly + philly", data = soda_data).fit()
print(store_FE_full_sample_reg.summary())

                            OLS Regression Results                            
Dep. Variable:          soda_quantity   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     14.03
Date:                Fri, 11 Feb 2022   Prob (F-statistic):           1.51e-13
Time:                        17:05:44   Log-Likelihood:                -32323.
No. Observations:                2452   AIC:                         6.466e+04
Df Residuals:                    2446   BIC:                         6.469e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

**A:** You need to add an interaction term between the after tax dummy and a dummy that selects stores outside Philly and within 6 miles of the border. This changes the interpretation of the coefficients as follows:
- *After-tax dummy coefficient*: is the difference in sales in stores outside Philly and more than 6 miles away after the tax relative to before the tax. 
- *The coefficient for the interaction term after_tax_dummy:philly*: is the differential change over time in sales in Philadelphia relative to the change over time in stores more than 6 miles away. 
- *The coefficient of the interaction term after_tax_dummy:near_phillyTRUE*: differential change over time in sales in stores within 6 miles of the city border relative to the change over time in stores more than 6 miles away of the city border.

**Conclusion:** This last regression confirms that the tax had a negative effect on sales. This controls for an overall decrease overtime in soda sales (confirmed by the fewer 898 bottles sold in stores that are more than 6 miles outside Philly in the period after the tax). The increase in sold bottles in stores outside Philly but close to the border suggests that there was a substitution effect of the tax, where people, instead of consuming less soda, simply drove just outside Philly to buy their sodas.